# TAOS2 - drifters1 campaign: yuco data

In [ ]:
import os
from glob import glob

%matplotlib inline
import matplotlib.pyplot as plt
import cmocean.cm as cm

import numpy as np
import pandas as pd
import xarray as xr
import hvplot.pandas  # noqa
from scipy import integrate

#
import pynsitu as pin
from pynsitu.maps import crs
import taos.sensors as se

In [ ]:
## taos
root_dir = "/Users/aponte/Current_Projects/taos/campagnes/taos2/"

# taos2-drifters0
campaign="taos2_drifters1"
yaml = f"drifters1/{campaign}.yaml"

cp = pin.Campaign(os.path.join(root_dir,yaml))

# plot all deployments
for label, deployment, platform, sensor, meta in cp.get_all_deployments():
    print(label, platform, sensor, deployment)
    
p = "yuco"

---

## yuco


In [ ]:
def read_yuco_csv(file):

    df = pd.read_csv(file)
    print(df.columns)

    new_col_names = [
        "timestamp", "time_seconds", "status", "surface_flag", 
         "latitude_ins", "longitude_ins", 
         "gps_flag", "latitude_gps", "longitude_gps", 
         "dvl_flag", "x_velocity", "y_velocity", "z_velocity", "depth",
         "altitude", "roll", "pitch", "yaw",
         "pressure_internal", "temperature_internal",
         "pressure_external", "temperature_external",
         "pressure_atmospheric", "battery",
         "legato_timestamp", "legato_time_seconds", "legato_count",
         "legato_conductivity", "legato_salinity", "legato_temperature_conductivity_cell",
         "legato_temperature", "legato_pressure",
         "legato_sea_pressure", "legato_depth",
         "legato_oxygen_concentration", "legato_ODO_temperature", "legato_ODO_phase", 
         "legato_turbidity", "legato_cholorophyll",
    ]

    df = df.rename(columns={p: n for p, n in zip(df.columns, new_col_names)})

    # process timestamps:
    df["time"] = pd.to_datetime(df['timestamp'],unit='s')
    df["legato_time"] = pd.to_datetime(df['legato_timestamp'],unit='s')
    df = df.set_index("time")    

    # turn flags into booleans
    df = df.replace({c: {'Y': 1, 'N': 0} for c in ["surface_flag", "gps_flag", "dvl_flag"]})
    
    # turn bad values into NaNs
    for c in ["longitude_ins", "latitude_ins", "longitude_gps", "latitude_gps"]:
        df.loc[:, c] = df.loc[:, c].replace(0,np.NaN)
    
    # rescale pressure in dbar, pressure is likely already in dbar
    #df["legato_pressure"] = df.loc[:, "legato_pressure"]*10
    #df["legato_sea_pressure"] = df.loc[:, "legato_sea_pressure"]*10
    
    # fix conductivity and pressure values which should be positive
    df["legato_conductivity"] =  df["legato_conductivity"].where(df.legato_conductivity>0)
    df["legato_sea_pressure"] = df["legato_sea_pressure"] - df["legato_sea_pressure"].min()
    
    # add total velocity
    df["h_velocity"] = np.sqrt(df.loc[:,"x_velocity"]**2 + df.loc[:,"y_velocity"]**2)
    df["r"] = integrate.cumulative_trapezoid(df.loc[:,"x_velocity"], x=df["time_seconds"], initial=0.)
    
    # select valid legato lines
    #dfl = df.loc[~pd.isna(df.loc[:,"legato_temperature"])]
    dfl = df.loc[~pd.isna(df["legato_temperature"]) & ~pd.isna(df["legato_conductivity"])]
    
    #legato_columns = [c for c in df.columns if "legato" in c]
    #df_legato = df.loc[:, legato_columns]
    
    # add columns to be compatible with pynsitu.seawater accessor
    dfl["temperature"] = dfl.loc[:,"legato_temperature"]
    dfl["pressure"] = dfl.loc[:,"legato_sea_pressure"]
    dfl["conductivity"] = dfl.loc[:,"legato_conductivity"]
    #dfl["longitude"] = dfl.loc[:,"longitude_ins"].fillna(dfl["longitude_ins"].median())
    #dfl["latitude"] = dfl.loc[:,"latitude_ins"].fillna(dfl["latitude_ins"].median())
    dfl["longitude"] = dfl["longitude_ins"].interpolate(limit_direction="both")
    dfl["latitude"] = dfl["latitude_ins"].interpolate(limit_direction="both")
    
    dfl.sw.update_eos()

    # select only valid values
    dfl = dfl.loc[~pd.isna(dfl["temperature"]) & ~pd.isna(dfl["salinity"])]
    
    return df, dfl

observed_variables = ["temperature", "legato_temperature", 'CT',
                      "conductivity", "legato_conductivity", 
                      "salinity", 'legato_salinity', 'SA',
                      'sigma0',
                      'legato_oxygen_concentration', 'legato_ODO_temperature', 'legato_ODO_phase',
                      'legato_cholorophyll', 
                     ]

### manually find deployment time intervals

In [ ]:
# browse deployments
run_dirs = sorted(glob(os.path.join(cp["path_raw"], "yuco/exports/YUCO-000D800D/*")))
runs = [r.split("/")[-1] for r in run_dirs]
run_files = {r:glob(os.path.join(d, "*.csv"))[0] for r, d in zip(runs, run_dirs)}

In [ ]:
i = 4 # 0 to 9
# 2 is not an actual run

r = runs[i]
print(r)

deployments = ["d0", "d1", None, "d2", "d3", "c4", "c5", "c6", "c7", "c8"]
d = deployments[i]

assert r==cp[p][d]["label"], "mismatch with yaml campaign file"

# load data
df, dfl = read_yuco_csv(run_files[r])

### refine deployment times



In [ ]:
# search for deployment times
_df = df.rename(columns=dict(longitude_ins="lon", latitude_ins="lat"))
_df.geo.plot_bokeh()

In [ ]:
(_df["legato_depth"].ffill().hvplot()
 +_df["legato_temperature"].ffill().hvplot()
 +_df["legato_conductivity"].ffill().hvplot()
).cols(1)

### identify surfacings and transects

In [ ]:
### split into seperate transects

# use conductivity to detect surfacings

# early parts of casts / spirals seems to be contaminated (conductivity)
# upcasts should be selected

T = dict(d0=[slice("2023/06/12 14:58:10", "2023/06/12 15:17:50"),
             slice("2023/06/12 15:17:50", "2023/06/12 15:18:30"), # @surface
             slice("2023/06/12 15:18:30", "2023/06/12 15:35:02"),
            ],
         d1=[slice("2023/06/12 15:50:45", "2023/06/12 16:09:04"),
            ],
         d2=[slice("2023/06/12 16:49:20", "2023/06/12 17:05:48"),
             slice("2023/06/12 17:05:48", "2023/06/12 17:06:25"), # @surface
             slice("2023/06/12 17:06:25", "2023/06/12 17:24:44"),
            ],
         d3=[slice("2023/06/12 17:45:36", "2023/06/12 18:01:15"),
             slice("2023/06/12 18:01:15", "2023/06/12 18:01:55"), # @surface
             slice("2023/06/12 18:01:55", "2023/06/12 18:21:38"),
            ],
         c4=[slice("2023/06/13 09:20:37", "2023/06/13 09:23:46"), # full CTD cast
            ],
         c5=[slice("2023/06/13 09:59:53", "2023/06/13 10:02:48"), # full CTD cast
            ],
         c6=[slice("2023/06/13 10:07:14", "2023/06/13 11:08:33"), # full CTD cast
            ],
         c7=[slice("2023/06/13 12:38:15", "2023/06/13 12:38:45"), # full CTD cast
            ],
         c8=[slice("2023/06/13 13:40:52", "2023/06/13 13:41:58"), # full CTD cast
            ],
        )

In [ ]:
#dfl["x_velocity"].hvplot()

### trim data

In [ ]:
# trim with yaml information
df = df.ts.trim(cp[p][d])
dfl = dfl.ts.trim(cp[p][d])

In [ ]:
dtl = (dfl.reset_index()["time"].diff().dropna()/pd.Timedelta("1s")).median()
print(f"Averaged time sampling = {dtl:.2f} s")

In [ ]:
# inspect velocity and depth

fig, axes = plt.subplots(2,1)

ax = axes[0]
dfl["h_velocity"].plot(ax=ax, label="velocity")
dfl["x_velocity"].plot(ax=ax, label="x - velocity")
dfl["y_velocity"].plot(ax=ax, label="y - velocity")
ax.legend()
ax.grid()
ax.set_xticklabels("")

ax = axes[1]
dfl["legato_depth"].plot(ax=ax, label="depth")
ax.grid()
ax.legend()

In [ ]:
def plot_on_map(df):
    #_df = df.rename(columns=dict(longitude_ins="longitude", latitude_ins="latitude"))
    _df = df.copy()
    lon_ref, lat_ref = cp["bounds"][0], cp["bounds"][2]
    _df.geo.set_projection_reference((lon_ref, lat_ref))
    _df.geo.compute_velocities(inplace=True)
    _df.geo.compute_accelerations(inplace=True)
    return _df.geo.plot_on_map(s=10, c="velocity", clim=(0,5), cmap="magma")

phv, coords = plot_on_map(dfl)
phv

### low pass filter and plot as a function of distance

In [ ]:
if d[0]=="d":
    rule = "5s"
    # mask surface points
    _df = dfl.copy()
    for v in observed_variables:
        _df.loc[:, v] = _df.loc[:, v].where(_df.depth>0.5)
    dfl_low = (_df
               .drop(columns="status")
               .resample(rule)
               .mean()
               .reset_index()
               .set_index("r")
    )    
elif d[0]=="c":
    #rule = "1s"
    dfl_low = dfl
    
# should treat cast particulary
# early parts of casts / spirals seems to be contaminated (conductivity)
# upcasts should be selected


In [ ]:
fig, axes = plt.subplots(4,1, figsize=(10,10))

ax = axes[0]
dfl.set_index("r")["temperature"].plot(ax=ax, label="raw")
dfl_low["temperature"].plot(ax=ax, label="temperature")
ax.grid()
ax.set_xlabel("")
ax.set_xticklabels("")
ax.set_ylabel("[degC]")
ax.legend()

ax = axes[1]
dfl.set_index("r")["salinity"].plot(ax=ax, label="raw")
dfl_low["salinity"].plot(ax=ax, label="salinity")
dfl_low["legato_salinity"].plot(ax=ax, label="legato_salinity")
ax.grid()
ax.set_xlabel("")
ax.set_xticklabels("")
ax.set_ylabel("[psu]")
ax.legend()

ax = axes[2]
dfl.set_index("r")["sigma0"].plot(ax=ax, label="raw")
dfl_low["sigma0"].plot(ax=ax, label="sigma0")
#dfl_low["legato_salinity"].plot(ax=ax, label="legato_salinity [psu]")
ax.grid()
ax.set_xlabel("r [m]")
ax.set_ylabel("[kg/m^3]")
ax.legend()


ax = axes[3]
dfl_low["depth"].plot(ax=ax, label="depth")
ax.grid()
ax.set_xlabel("")
ax.set_xticklabels("")
ax.set_ylabel("[m]")
ax.legend()


In [ ]:
# do not execute for casts
if d[0]!="c":

    phv, coords = plot_on_map(dfl_low.set_index("time"))
    
    dfl_low.loc[:, "transect"] = 0
    _t = dfl_low.loc[:,"time"]
    for i, t in enumerate(T[d]):
        dfl_low.loc[ _t>t.start ,  "transect"] = i
    phv

In [ ]:
fig, axes = plt.subplots(3,1,)

ax = axes[0]
for t in dfl_low.transect.unique():
    dfl_low.loc[dfl_low.transect==t]["temperature"].plot(ax=ax, label=str(int(t)))
ax.grid()
ax.set_xlabel("")
ax.set_xticklabels("")
ax.set_ylabel("[degC]")
ax.legend()

ax = axes[1]
for t in dfl_low.transect.unique():
    dfl_low.loc[dfl_low.transect==t]["salinity"].plot(ax=ax, label=str(int(t)))
#dfl_low["salinity"].plot(ax=ax, label="salinity")
#dfl_low["legato_salinity"].plot(ax=ax, label="legato_salinity")
ax.grid()
ax.set_xlabel("")
ax.set_xticklabels("")
ax.set_ylabel("[psu]")
ax.legend()

ax = axes[2]
for t in dfl_low.transect.unique():
    dfl_low.loc[dfl_low.transect==t]["sigma0"].plot(ax=ax, label=str(int(t)))
#dfl_low["sigma0"].plot(ax=ax, label="sigma0")
#dfl_low["legato_salinity"].plot(ax=ax, label="legato_salinity [psu]")
ax.grid()
ax.set_xlabel("r [m]")
ax.set_ylabel("[kg/m^3]")
ax.legend()


In [ ]:
_df = dfl_low
_df.to_xarray().to_netcdf(os.path.join(cp["path_processed"], f'{p}_{d}.nc'), mode="w")